In [ ]:
!nvidia-smi

В данных 54 магазина и 33 семейства продуктов.                     
Временной ряд тренировочных данных с 01.01.2013 по 15.08.2017.              
Временной ряд тестовых данных для submition составляет 16 дней после последней даты тренировочных данных: с 16.08.2017 по 31.08.2017.                   
                                 
Нужно составить прогноз продаж для каждого из семейств продуктов в каждом из магазинов.     
                             
В отдельном ноутбуке проведен Feature engineering и сформированы необходимые датасеты.
                                
**Текущие датасеты**                   
1. featured_data - объединенные данные test и train с новыми признаками.
2. zero_prediction - данные тех товаров, которые не продавались в конкретном магазине с начала 2013 года, исходя из чего можно предположить, что данные товары не будут продаваться в ближайшие 16 дней. Этот датасет мы будем объединять с предсказанными данными перед отправкой в submit. 

In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import warnings

# PACF - ACF
# ------------------------------------------------------
import statsmodels.api as sm

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')
from warnings import simplefilter
%matplotlib inline
from tqdm.auto import tqdm

# 1. Import Data

In [ ]:
# Import
df = pd.read_csv("../input/featured-data/featured_data.csv")
zero_prediction = pd.read_csv("../input/zero-prediction/zero_prediction.csv")

# Datetime
df["date"] = pd.to_datetime(df.date)

zero_prediction = zero_prediction.set_index(['store_nbr', 'family', 'date']).sort_index()

In [ ]:
d_train = df[df.date<'2017-08-16'].copy()
d_test = df[df.date>='2017-08-16'].copy()

# Критерий Дикки-Фуллера

In [ ]:
# # Подсчет критерия Дикки-Фуллера для каждого из семейства товаров в каждом из магазинов

# # для экономии расчетного времени данный код закомментирован
# # не стационарные ряды собраны в fuller_not_stat

# # ряды, где недостаточно данных для подсчета критерия Дикки-Фуллера, полностью совпадают с 
# # комбинациями из zero_prediction.

# fuller_not_stat = pd.DataFrame(columns = ['store_nbr', 'family'])
# small_data = pd.DataFrame(columns = ['store_nbr', 'family'])

# for num, store in enumerate(tqdm(d_train.store_nbr.unique())):
#     for numf, fam in enumerate(tqdm(d_train.family.unique())):
#         a=d_train[(d_train.store_nbr == store) & (d_train.family == fam)]
#         try:
#             adftest = sm.tsa.adfuller(a.sales)
#             print('Магазин: ', store, 'семейство товаров: ', fam)
#             print('adf: ', adftest[0])
#             print('p-value: ', adftest[1])
#             print('Critical values: ', adftest[4])
#             if adftest[0]> adftest[4]['5%']: 
#                 print('есть единичные корни, ряд не стационарен')
#                 fuller_not_stat = fuller_not_stat.append({'store_nbr':store, 'family':fam}, ignore_index=True)
#                 print('___'*20)
#             else:
#                 print('единичных корней нет, ряд стационарен')
#                 print('___'*20)
#         except ValueError:
#                 print('Магазин: ', store, 'семейство товаров: ', fam)
#                 print('Количество данных:', len(a))
#                 small_data = small_data.append({'store_nbr':store, 'family':fam}, ignore_index=True)
#                 print('___'*20)
#                 continue
                
# # OUTPUT EXAMPLE:                
# # Магазин:  1 семейство товаров:  AUTOMOTIVE
# # adf:  -4.078623538152452
# # p-value:  0.001050394088497745
# # Critical values:  {'1%': -3.4342954463097706, '5%': -2.8632826898390484, '10%': -2.5676977663666714}
# # единичных корней нет, ряд стационарен            
# fuller_not_stat.to_csv('fuller_not_stat.csv', index=False)
# small_data.to_csv('small_data.csv', index=False)

Для нестационарных рядов оказалось достаточно первой разности для приведения ряда к стационарному.

In [ ]:
fuller_result = pd.read_csv("../input/fuller-result/fuller_not_stat.csv")

In [ ]:
# Для нестационарных рядов делаем список из комбинаций store + family
fuller_list = fuller_result.values
s_fuller_list=[]
for i in range(fuller_list.shape[0]):
    s_fuller_list.append(str(fuller_list[i][0]) + ',' + fuller_list[i][1])

# Скользящее среднее

In [ ]:
# for num, store in enumerate(tqdm(d_train.store_nbr.unique())):
#     for numf, fam in enumerate(tqdm(d_train.family.unique())):
#         try:
#             a=d_train[(d_train.store_nbr == store) & (d_train.family == fam)].copy()
#             a=a.sales
#             b = str(store)+','+fam
#             if b in s_fuller_list:
#                 a = a.diff().dropna() # если ряд не стационарный, то берем для него первую разность
#             # d_train.loc[(d_train.store_nbr == store) & (d_train.family == fam),"moving_avg"] = a.rolling(7).mean()
#             d_test.loc[(d_test.store_nbr == store) & (d_test.family == fam),'moving_avg']=a.rolling(7).mean().iloc[-1]
#             #print('store:',store,'fam:',fam)
#         except:
#             continue

### Submit for moving_average

In [ ]:
# d_test = d_test.set_index(['store_nbr', 'family', 'date']).sort_index()
# for i in zero_prediction.index:
#     d_test.loc[i,'moving_avg']=0

In [ ]:
# d_test=d_test.reset_index()
# y_submit = d_test[['id','moving_avg']].copy()
# y_submit.columns = ['id', 'sales']
# y_submit.to_csv('submission_moving_avg.csv', index=False)

In [ ]:
# y_submit.tail()

**Результат на kaggle 1.90138**

In [ ]:
# gc.collect()

# ARIMA

ARIMA зависит от значений p, d, q:
   - p = количество лагов, параметр AR;
   - d = порядок разности;
   - q = количество запаздывающих ошибок прогноза, параметр MA.
aic (информационный критерий Акаике):
- более низкий aic указывает на лучшую модель;
- aic выбирет простые модели с более низким порядком.

Перебор параметров оказался 

In [ ]:
# ps = range(0, 10)
# d=1
# qs = range(0, 10)
# # Ps = range(0, 5)
# # D=1
# # Qs = range(0, 1)

# from itertools import product

# parameters = product(ps, qs)#, Ps, Qs)
# parameters_list = list(parameters)
# len(parameters_list)

In [ ]:
# %%time
# # Подбор параметров p и q для модели ARIMA
# all_result_table = pd.DataFrame()
# simplefilter("ignore")  # ignore warnings to clean up output cells

# for num, store in enumerate(tqdm(d_train.store_nbr.unique())):
#     for numf, fam in enumerate(tqdm(d_train.family.unique())):
#         a=d_train[(d_train.store_nbr == store) & (d_train.family == fam)].copy()
#         a=a.set_index('date')
#         b = str(store)+','+fam
#         if b in s_fuller_list:
#             a.sales = a.sales.diff().dropna() # если ряд не стационарный, то берем для него первую разность
#         results = []
#         best_aic = float("inf")
#         if len(a)!=0:
#             for param in tqdm(parameters_list):
#                 try:
#                     model=sm.tsa.arima.ARIMA(a.sales, order=(param[0], d, param[1]), enforce_stationarity=False).fit()#, 
#                                         #seasonal_order=(param[3], D, param[3], 24*7)).fit(disp=0)
#                 #выводим параметры, на которых модель не обучается и переходим к следующему набору
#                 except ValueError:
#                     print('wrong parameters:', param)
#                     continue
#                 aic = model.aic
#                 #сохраняем лучшую модель, aic, параметры
#                 if aic < best_aic:
#                     best_model = model
#                     best_aic = aic
#                     best_param = param
#                 results.append([param, model.aic])

#                 # warnings.filterwarnings('default')

#             result_table = pd.DataFrame(results)
#             result_table.columns = ['parameters', 'aic']
#             result_table = result_table.sort_values(by = 'aic', ascending=True)[:1]
#             result_table['store_nbr'] = store
#             result_table['family'] = fam
#             all_result_table= all_result_table.append(result_table, ignore_index=True)
#             print('best param: ', result_table)
#             best_model = sm.tsa.arima.ARIMA(a.sales, order=(param[0], d, param[1])).fit()#, seasonal_order=(param[3], D, param[3], 24*7)).fit(disp=0)
#             # d_train.loc[(d_train.store_nbr == store) & (d_train.family == fam),"arima_model"] = np.array(best_model.fittedvalues)
#             d_test.loc[(d_test.store_nbr == store) & (d_test.family == fam),'arima_model']=np.array(best_model.predict(start = a.shape[0], end = a.shape[0]+15))
#         else:
#             continue
            
# print(all_result_table)

### Submit for ARIMA

In [ ]:
# d_test = d_test.set_index(['store_nbr', 'family', 'date']).sort_index()
# for i in zero_prediction.index:
#     d_test.loc[i,'arima_model']=0

In [ ]:
# d_test = d_test.reset_index()
# y_submit = d_test[['id','arima_model']].copy()
# y_submit.columns = ['id', 'sales']
# y_submit.to_csv('submission_arima_model.csv', index=False)

In [ ]:
# y_submit.tail()

In [ ]:
# gc.collect()

# Модель Хольта-Винтерса

In [ ]:
# from statsmodels.tsa.api import ExponentialSmoothing

In [ ]:
# for num, store in enumerate(tqdm(d_train.store_nbr.unique())):
#     for numf, fam in enumerate(tqdm(d_train.family.unique())):
#         a = d_train[(d_train.store_nbr == store) & (d_train.family == fam)].copy()
        
#         if len(a)!=0:
#             model_HW = ExponentialSmoothing(a.sales, seasonal_periods=7, trend='add', seasonal='add').fit()
#             # d_train.loc[(d_train.store_nbr == store) & (d_train.family == fam),'HW_pred']=np.array(model_HW.fittedvalues)
#             d_test.loc[(d_test.store_nbr == store) & (d_test.family == fam),'HW_pred']=np.array(model_HW.forecast(16))
#             #print('store:', store, 'fam:', fam)

### Submit for Holt-Winters model

In [ ]:
# #d_test = d_test.reset_index().set_index(['store_nbr', 'family', 'date']).sort_index()
# d_test = d_test.set_index(['store_nbr', 'family', 'date']).sort_index()
# for i in zero_prediction.index:
#     d_test.loc[i,'HW_pred']=0

In [ ]:
# d_test=d_test.reset_index()
# HW_submit = d_test[['id','HW_pred']].copy()
# HW_submit['HW_pred'] = HW_submit.HW_pred.clip(0)

In [ ]:
# # d_test=d_test.reset_index()
# # HW_submit = d_test[['id','HW_pred']].copy()
# HW_submit.columns = ['id', 'sales']
# HW_submit.to_csv('submission_HW_clip.csv', index=False)

In [ ]:
# HW_submit.head(5)

**Результат на kaggle 0.43267**

In [ ]:
# gc.collect()

# Linear Regression

Сделаем отступление от принятой модели вычисления предсказания и возьмем среднее по дням вне зависимостри от семейства или магазина. Изучим полученный ряд на тренд и сезонность, построим периодограмму.

### Тренд для обобщенного ряда

In [ ]:
# from pathlib import Path
# from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression

In [ ]:
# simplefilter("ignore")  # ignore warnings to clean up output cells

# # Set Matplotlib defaults
# plt.style.use("seaborn-whitegrid")
# plt.rc("figure", autolayout=True, figsize=(11, 5))
# plt.rc(
#     "axes",
#     labelweight="bold",
#     labelsize="large",
#     titleweight="bold",
#     titlesize=14,
#     titlepad=10,
# )
# plot_params = dict(
#     color="0.75",
#     style=".-",
#     markeredgecolor="0.25",
#     markerfacecolor="0.25",
#     legend=False,
# )
# %config InlineBackend.figure_format = 'retina'

# # Группируем по дню 
# avg_sales = d_train.groupby('date').agg({'sales': 'mean'}).reset_index()
# avg_sales = avg_sales.set_index('date').to_period("D")


# dp = DeterministicProcess(
#     index=avg_sales.index,  # dates from the training data
#     constant=True,       # dummy feature for the bias (y_intercept)
#     order=1,             # the time dummy (trend)
#     drop=True,           # drop terms if necessary to avoid collinearity
# )
# # `in_sample` creates features for the dates given in the `index` argument
# X = dp.in_sample()
# y = avg_sales["sales"]  # the target

# model = LinearRegression(fit_intercept=False)
# model.fit(X, y)
# y_pred = pd.Series(model.predict(X), index=X.index)

# ax = avg_sales.plot(style=".", color="0.5", title="sales - Linear Trend")
# _ = y_pred.plot(ax=ax, linewidth=3, label="Trend")

### Сезонность для обобщенного ряда

In [ ]:
# def seasonal_plot(X, y, period, freq, ax=None):
#     if ax is None:
#         _, ax = plt.subplots()
#     palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
#     ax = sns.lineplot(
#         x=freq,
#         y=y,
#         hue=period,
#         data=X,
#         ci=False,
#         ax=ax,
#         palette=palette,
#         legend=False,
#     )
#     ax.set_title(f"Seasonal Plot ({period}/{freq})")
#     for line, name in zip(ax.lines, X[period].unique()):
#         y_ = line.get_ydata()[-1]
#         ax.annotate(
#             name,
#             xy=(1, y_),
#             xytext=(6, 0),
#             color=line.get_color(),
#             xycoords=ax.get_yaxis_transform(),
#             textcoords="offset points",
#             size=14,
#             va="center",
#         )
#     return ax


# def plot_periodogram(ts, detrend='linear', ax=None):
#     from scipy.signal import periodogram
#     fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
#     freqencies, spectrum = periodogram(
#         ts,
#         fs=fs,
#         detrend=detrend,
#         window="boxcar",
#         scaling='spectrum',
#     )
#     if ax is None:
#         _, ax = plt.subplots()
#     ax.step(freqencies, spectrum, color="purple")
#     ax.set_xscale("log")
#     ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
#     ax.set_xticklabels(
#         [
#             "Annual (1)",
#             "Semiannual (2)",
#             "Quarterly (4)",
#             "Bimonthly (6)",
#             "Monthly (12)",
#             "Biweekly (26)",
#             "Weekly (52)",
#             "Semiweekly (104)",
#         ],
#         rotation=30,
#     )
#     ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
#     ax.set_ylabel("Variance")
#     ax.set_title("Periodogram")
#     return ax

In [ ]:
# X_timefit = avg_sales.copy()

# # days within a week
# X_timefit['day'] = X_timefit.index.dayofweek # the x-axis (freq)
# X_timefit['week'] = X_timefit.index.week # the seasonal period (period)

# # days within a year
# X_timefit['dayofyear'] = X_timefit.index.dayofyear
# X_timefit['year'] = X_timefit.index.year

# fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(11, 6))
# seasonal_plot(X_timefit, y="sales", period="week", freq="day", ax=ax0)
# seasonal_plot(X_timefit, y="sales", period="year", freq="dayofyear", ax=ax1);

In [ ]:
# y_detrend = y - y_pred 

# fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))
# ax1 = plot_periodogram(y, ax=ax1)
# ax1.set_title("Product Sales Frequency Components")
# ax2 = plot_periodogram(y_detrend, ax=ax2);
# ax2.set_title("Detrended");

Прослеживается сильный недельный сезон и более слабый годовой сезон. Недельный сезон мы будем моделировать с помощью индикаторов, а годовой сезон — с помощью функций Фурье. Периодограмма затихает между раз Bimonthly (6) и Monthly (12),будем используем 10 пар Фурье.

Полученный рузельтат будем применять в разрезе магазинов и семейств товаров.

In [ ]:
# # В нашем датасете остались категориальные признаки,
# # которые необходимо закодировать в числовые.

# lab_enc_col = ['city','state','type','cluster']
# # Label Encoding
# for column in lab_enc_col:
#     df[column] = df[column].astype('category').cat.codes
        
# # One-Hot Encoding c get_dummies.
# df = pd.get_dummies(df, columns=lab_enc_col, dummy_na=False)

# # Раздеряем на test и train
# d_train = df[df.date<'2017-08-16'].copy()
# d_test = df[df.date>='2017-08-16'].copy()

In [ ]:
d_train=d_train.drop(['city','state','type','cluster'], axis=1)
d_test=d_test.drop(['city','state','type','cluster'], axis=1)

In [ ]:
d_test['lr_model'] = 0
# fourier = CalendarFourier(freq="A", order=10) 
for num, store in enumerate(tqdm(d_train.store_nbr.unique())):
    for numf, fam in enumerate(tqdm(d_train.family.unique())):
        a = d_train[(d_train.store_nbr == store) & (d_train.family == fam)].copy()
        a = a.set_index('date').to_period("D")
        b = str(store)+','+fam
        
        if len(a)!=0:
            if b in s_fuller_list:
                a.sales = a.sales.diff().fillna(0)
                
#             dp = DeterministicProcess(
#                 index=a.index,
#                 constant=True,   # dummy feature for bias (y-intercept)
#                 order=1,         # trend ( order 1 means linear)
#                 seasonal=True,   # weekly seasonality (indicators)
#                 additional_terms=[fourier], # annual seasonality
#                 drop=True,       # drop terms to avoid collinearity
#             )

#             X = dp.in_sample() # create features for dates in tunnel.index
            
#             X = X.join(a.drop(['family','store_nbr','id','sales','year'],axis=1))
            X = a.drop(['family','store_nbr','id','sales','year'],axis=1).copy()
            y = a.sales

            model = LinearRegression(fit_intercept=False)
            model.fit(X, y)

#             X_fore = dp.out_of_sample(16)
            a_fore = d_test[(d_test.store_nbr == store) & (d_test.family == fam)].copy()
            a_fore = a_fore.set_index('date').to_period("D")
#             X_fore = X_fore.join(a_fore.drop(['family','store_nbr','id','sales','year','lr_model'],axis=1))
            X_fore = a_fore.drop(['family','store_nbr','id','sales','year','lr_model'],axis=1).copy()
            d_test.loc[(d_test.store_nbr == store) & (d_test.family == fam), 'lr_model'] = model.predict(X_fore)

### Submit for Linear Regression

In [ ]:
d_test = d_test.set_index(['store_nbr', 'family', 'date']).sort_index()
for i in zero_prediction.index:
    d_test.loc[i,'lr_model']=0

In [ ]:
d_test=d_test.reset_index()
lr_submit = d_test[['id','lr_model']].copy()
lr_submit['lr_model'] = lr_submit.lr_model.clip(0)
lr_submit.columns = ['id', 'sales']
lr_submit.to_csv('submission_lr_clip.csv', index=False)

In [ ]:
lr_submit.tail()

**Результат на kaggle 2.00271**

In [ ]:
gc.collect()

# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
d_test['xgb_model'] = 0
# fourier = CalendarFourier(freq="A", order=10) 
for num, store in enumerate(tqdm(d_train.store_nbr.unique())):
    for numf, fam in enumerate(tqdm(d_train.family.unique())):
        a = d_train[(d_train.store_nbr == store) & (d_train.family == fam)].copy()
        a = a.set_index('date').to_period("D")
        b = str(store)+','+fam
        
        if len(a)!=0:
            if b in s_fuller_list:
                a.sales = a.sales.diff().fillna(0)
                
#             dp = DeterministicProcess(
#                 index=a.index,
#                 constant=True,   # dummy feature for bias (y-intercept)
#                 order=1,         # trend ( order 1 means linear)
#                 seasonal=True,   # weekly seasonality (indicators)
#                 additional_terms=[fourier], # annual seasonality
#                 drop=True,       # drop terms to avoid collinearity
#             )

#             X = dp.in_sample() # create features for dates in tunnel.index
#             X_train = X.join(a.drop(['family','store_nbr','id','sales','year'],axis=1))# !!! удалить lr_model
            X_train = a.drop(['family','store_nbr','id','sales','year'],axis=1).copy()
            y = a.sales
            
#             X_fore = dp.out_of_sample(16)
            a_fore = d_test[(d_test.store_nbr == store) & (d_test.family == fam)].copy()
            a_fore = a_fore.set_index('date').to_period("D")
#             X_fore = X_fore.join(a_fore.drop(['family','store_nbr','id','sales','year','xgb_model'],axis=1)) # !!!
            X_fore = a_fore.drop(['family','store_nbr','id','sales','year','lr_model','xgb_model'],axis=1).copy()
            
            dtrain = xgb.DMatrix(X_train, label=y)
            dtest = xgb.DMatrix(X_fore)
            
            # задаём параметры
            params = {
                'objective': 'reg:squarederror',
                'booster':'gblinear'
            }
            trees = 100
            
            # прогоняем на кросс-валидации с метрикой rmse
            cv = xgb.cv(params, dtrain, metrics = ('rmse'), verbose_eval=False, nfold=5, show_stdv=False, num_boost_round=trees)
            # print('store: ', store, 'fam: ', fam)
            # обучаем xgboost с оптимальным числом деревьев, подобранным на кросс-валидации
            bst = xgb.train(params, dtrain, num_boost_round=cv['test-rmse-mean'].argmin())

            d_test.loc[(d_test.store_nbr == store) & (d_test.family == fam), 'xgb_model'] = bst.predict(dtest)

### Submit for XGBoost

In [ ]:
d_test = d_test.set_index(['store_nbr', 'family', 'date']).sort_index()
for i in zero_prediction.index:
    d_test.loc[i,'xgb_model']=0

In [ ]:
d_test=d_test.reset_index()
xgb_submit = d_test[['id','xgb_model']].copy()
xgb_submit['xgb_model'] = xgb_submit.xgb_model.clip(0)
xgb_submit.columns = ['id', 'sales']
xgb_submit.to_csv('submission_xgb_clip.csv', index=False)

In [ ]:
xgb_submit.tail()